In [0]:
# train on multilple machines
# Move all gradient to a machne
# aggregate the gradient on that machine and then get it back to our machine

In [0]:
! pip install syft

     |████████████████████████████████| 194kB 2.8MB/s 
     |████████████████████████████████| 1.4MB 33.0MB/s 
     |████████████████████████████████| 71kB 26.0MB/s 
     |████████████████████████████████| 389kB 46.6MB/s 
     |████████████████████████████████| 450kB 49.9MB/s 
     |████████████████████████████████| 204kB 42.8MB/s 
     |████████████████████████████████| 276kB 45.0MB/s 
     |████████████████████████████████| 51kB 22.7MB/s 
     |████████████████████████████████| 122kB 51.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/ad/9a/f4/3105b5209674ac77fcca7fede95184c62a95df0196888e0e76
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built zstd pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [0]:
import torch as th
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import SubsetRandomSampler
from torchvision import datasets, transforms

import syft as sy

W0623 14:38:37.769871 140553504298880 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0-rc1.so'
W0623 14:38:37.794982 140553504298880 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
hook = sy.TorchHook(th)

In [0]:
# federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
#     datasets.CIFAR10('../data', train=True, download=True,
#                    transform=transforms.Compose([
#                        transforms.ToTensor(),
#                        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
#                    ]))
#     .federate((bob, alice)), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
#     batch_size=args.batch_size, shuffle=True, **kwargs)

In [0]:
def create_data_loaders (num_training_loaders):

    # Define a transform to normalize the data
    transform=transforms.Compose([transforms.ToTensor(),
                                  transforms.Normalize((0.1307,), (0.3081,))])
    # Download and load the training data
    mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    # Set up the trainloaders list
    trainloaders = []
    data_per_set = int(len(mnist_trainset) / num_training_loaders)
    for i in range(num_training_loaders):
        train_indices = range(i*data_per_set,i*data_per_set+data_per_set)
        trainloaders.append(torch.utils.data.DataLoader(mnist_trainset, 
                                                         batch_size=32, 
                                                         sampler=SubsetRandomSampler(train_indices)))

    # Download and load the test data
    mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
    mnist_test_train = Subset(mnist_testset,range(9000))
    mnist_test_test = Subset(mnist_testset,range(9000,10001))
    # Set up the testloader
    # Use for train student model
    testtrainloader = torch.utils.data.DataLoader(mnist_test_train, batch_size=32, shuffle=False)
    testtrain_targets = mnist_testset.test_labels[:9000]
    
    # Use for evaluate the student model
    testtestloader = torch.utils.data.DataLoader(mnist_test_test, batch_size=1000, shuffle=False)
    testtest_targets = mnist_testset.test_labels[9000:]

    return trainloaders, testtrainloader,testtestloader, testtrain_targets, testtest_targets

In [0]:
def create_federated_data_loaders(num_training_loaders,workers):

    # Define a transform to normalize the data
    transform=transforms.Compose([transforms.ToTensor(),
                                  transforms.Normalize((0.1307,), (0.3081,))])
    # Download and load the training data
    mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    # Set up the trainloaders list
    trainloaders = []
    data_per_set = int(len(mnist_trainset) / num_training_loaders)
    for i in range(num_training_loaders):
        train_indices = range(i*data_per_set,i*data_per_set+data_per_set)
#         trainloaders.append(torch.utils.data.DataLoader(mnist_trainset, 
#                                                          batch_size=32, 
#                                                          sampler=SubsetRandomSampler(train_indices)))
        trainloaders.append(sy.FederatedDataLoader(mnist_trainset, 
                                                         batch_size=32, 
                                                         sampler=SubsetRandomSampler(train_indices))).federate(workers)

    # Download and load the test data
    mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
    # Set up the testloader
    
    testtrainloader = sy.FederatedDataLoader(mnist_testset, batch_size=1000, shuffle=False,
                                                 sampler=SubsetRandomSampler(range(9000))).federate(workers)

    testtestloader = sy.FederatedDataLoader(mnist_testset, batch_size=1000, shuffle=False,
                                                 sampler=SubsetRandomSampler(range(9000,10001))).federate(workers)
    test_targets = mnist_testset.test_labels

    return trainloaders, testtrainloader,testtestloader, test_targets

In [0]:
bob, alice, secure = sy.VirtualWorker(hook,id='bob'), sy.VirtualWorker(hook,id='alice'), sy.VirtualWorker(hook,id='secure')

In [0]:
train_loader, test_loader = create_federated_data_loaders(2,)

TypeError: ignored

In [0]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784,256)
        self.fc2 = nn.Linear(256,64)
        self.fc3 = nn.Linear(64,10)
        
    def forward(self,x):
        x = x.view(x.shape[0],-1)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.log_softmax(self.fc3(x),dim=1)
    
        return x
        

In [0]:
# create a model and send it to both machines
# compute anything on those 2 machines 
# at the end send those 2 model to secure workder
# averange the weights and bias

In [0]:
def train(model, machines, secure, dataset,epochs=4):
    criterion = nn.NLLLoss() 
    models = [] # store the model on all machines
    for machine in machines : 
        model_ = model.copy().send(machine) # model on the remote machine
        optimizer_ = optim.Adam(model_.parameters(),lr=1e-3)
        for i in range(epochs):
            running_loss = 0
            for data, target in dataset:
                _data = data.send(machine)
                _target = target.send(machine)

                optimizer_.zero_grad()
                pred = model_(_data)
                loss = criterion(pred,_target)
                loss.backward()
                optimizer_.step()

#                 model = model.get()
                running_loss +=loss.get()
            
            print(f"Epoch {i} : {running_loss/len(dataset)}")
        model_.move(secure) # send the secure worker
        models.append(model_)
    return models # later to aggregare gradieint

In [0]:
model = Net()

In [0]:
models = train(model,(bob,alice),secure,train_loader)

In [0]:
# aggregate the model

In [0]:
with th.no_grad():
    for model_layer,bob_layer,alice_layer in zip(model.children(),models[0].children(),models[1].children()):
        model_layer.weight.set_(((bob_layer.weight.data + alice_layer.weight.data)/2).get())